# Need for Graph
The key characteristic in LCEL chain is that the data flows in one direction. The prompt's output is piped to the model, and the model's output is piped to the parser. This structure is a Directed Acyclic Graph (DAG). It's "directed" because the data flows in a set direction, and "acyclic" because it contains no loops or cycles. Think of it as a factory assembly line: an item moves from one station to the next, but it never goes backward.

**No Self correction, No Dynamic Re-Planning, Inability to Cycle**


## The Solution: State and Cycles 🔄

**State**: Instead of passing a single output from one step to the next, all steps (called nodes) in a LangGraph share access to a central state object. Each node can read from this state and return updates to it. This state could contain the original input, chat history, intermediate steps, and tool outputs.

**Cycles**: LangGraph allows you to define edges that connect nodes. Crucially, these edges can be conditional. This means you can create rules that say, "After running the tool node, check the state. If there was an error, loop back to the LLM node. If it was successful, proceed to the final answer node." This allows us to explicitly create the loops that agentic behavior requires.

In [2]:
import os
from typing import TypedDict, Annotated
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, AnyMessage, ToolMessage
from langgraph.graph import StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages

from dotenv import load_dotenv
load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")

The **state** is the heart of a LangGraph application. It's a central object that every node in the graph can read from and write to. We define its structure using a Python TypedDict

simple chatbot to learn basics


In [3]:
class GraphState(TypedDict):
    """represent the state of our graph
    attributes:
    messages: list of messages in the chat"""
    messages:Annotated[list,add_messages] #annoted is a feature that lets you add metadata to a variable, 
    # like a comment or description.

Annotated[list, add_messages] means:
“This field is a list, and it should be treated specially using the add_messages logic.”

TypedDict: This gives us a structured, predictable state object.

Nodes are the "workers" of the graph. They are simple Python functions that perform a task. Each node function receives the current state as its only argument and must return a dictionary with the values to update in the state.

In [4]:
model= GoogleGenerativeAI(model="gemini-2.0-flash", api_key=google_api_key)

def call_model(state:GraphState):
    """invokes model with current state"""
    messages=state["messages"]
    response=model.invoke(messages)
    return {"messages":[response]}
# call_model() is a node that takes the current state of the graph and invokes the model with it. 
# It returns a new state with the model's response added to the messages list.

### We'll create a StateGraph instance and define the nodes and the edges (the connections) between them.

In LangGraph, a checkpointer is used to save and restore the state of your graph as it runs
<ul>
<li>Resuming: If your process stops or crashes, you can resume from the last saved state.</li>
<li>Debugging: You can inspect previous states to see how your data changed.</li>
<li>Persistence: You can keep a record of all the steps and messages in your workflow.</li>
</ul>

In [5]:
graph_builder= StateGraph(GraphState)

graph_builder.add_node("llm", call_model,description="Call the LLM with the current state")

graph_builder.set_entry_point("llm")
#simple edge that loops back to the LLM node
graph_builder.set_finish_point("llm")

memory = MemorySaver()
graph=graph_builder.compile(checkpointer=memory)

Since this is a conversational chatbot we will run the graph in loop

We create a config dictionary. The thread_id is crucial for the checkpointer to know which conversation's state to load and save.

In [6]:
config={"configurable":{"thread_id":"first_convo"}} #this is a configuration for the graph, it can be used to pass parameters to the graph

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Exiting the chatbot.")
        break
    events=graph.stream(
        {"messages":[HumanMessage(content=user_input)]},
        config=config
    )
    # The 'values' attribute of the event contains the output of the node
    
    #event->{'llm': {'messages': ['Hello! How can I assist you?']}}
    for event in events:
        response = event['llm']['messages'][-1]
        print(response)
        # This prints the last message in the messages list, which is the model's response.

Hi there! How can I help you today?
Hi Alex! It's nice to meet you. How can I help you today?
Okay, so you want to know who "the boss" is. To help me answer that, I need a little more context. Are you asking:

*   **In general?** Like, what is a boss?
*   **About a specific company?** If so, which company?
*   **About the TV show "Who's the Boss?"**

Tell me more about what you're looking for!
Okay, I understand you might be a little confused. Let's try this:

I'm here to help you find information or answer questions. You asked "whos the boss".

To give you the best answer, I need to know what you're asking about. Are you interested in:

*   **Finding out who the boss is at a particular company?**
*   **Learning about what a boss *is* in general?**
*   **Something else entirely?**

Just tell me what you're thinking!
Exiting the chatbot.
